In [55]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


from tabulate import tabulate

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
import pathlib
dataset_dir = "drive/MyDrive/Lung Disease Detection/combined_train"

In [58]:
batch_size = 32
img_height = 256
img_width = 256

In [59]:
from numpy import testing
train_ds = tf.keras.utils.image_dataset_from_directory(
                dataset_dir,
                validation_split=0.2,
                subset="training",
                seed=123,
                image_size=(img_height, img_width),
                batch_size=batch_size)

Found 2935 files belonging to 4 classes.
Using 2348 files for training.


In [60]:
val_ds = tf.keras.utils.image_dataset_from_directory(
                dataset_dir,
                validation_split=0.2,
                subset="validation",
                seed=123,
                image_size=(img_height, img_width),
                batch_size=batch_size)

Found 2935 files belonging to 4 classes.
Using 587 files for validation.


In [61]:
class_names = train_ds.class_names
print(class_names)

['diseased_ctscan', 'diseased_xray', 'not_diseased_ctscan', 'not_diseased_xray']


In [62]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 256, 256, 3)
(32,)


In [63]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [64]:
normalization_layer = layers.Rescaling(1./255)

In [65]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [66]:
resize = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(img_height,img_width)
    
])

In [67]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [68]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resize,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

In [69]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [70]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_9 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 sequential_9 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d_38 (Conv2D)          (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 128, 128, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_39 (Conv2D)          (None, 128, 128, 16)      2320      
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                

In [71]:
epochs=20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


74/74 [==============================] - 179s 2s/step - loss: 0.8480 - accuracy: 0.5830 - val_loss: 0.5614 - val_accuracy: 0.7462
Epoch 2/20
74/74 [==============================] - 107s 1s/step - loss: 0.3422 - accuracy: 0.8739 - val_loss: 0.2378 - val_accuracy: 0.8995
Epoch 3/20
74/74 [==============================] - 106s 1s/step - loss: 0.2352 - accuracy: 0.9174 - val_loss: 0.1860 - val_accuracy: 0.9319
Epoch 4/20
74/74 [==============================] - 106s 1s/step - loss: 0.1706 - accuracy: 0.9378 - val_loss: 0.1991 - val_accuracy: 0.9267
Epoch 5/20
74/74 [==============================] - 106s 1s/step - loss: 0.1641 - accuracy: 0.9451 - val_loss: 0.1359 - val_accuracy: 0.9489
Epoch 6/20
74/74 [==============================] - 107s 1s/step - loss: 0.0957 - accuracy: 0.9689 - val_loss: 0.1132 - val_accuracy: 0.9591
Epoch 7/20
74/74 [==============================] - 106s 1s/step - loss: 0.0745 - accuracy: 0.9719 - val_loss: 0.1249 - val_accuracy: 0.9489
Epoch 8/20
74/74 [======

In [72]:
test_dir = "drive/MyDrive/Lung Disease Detection/combined_test"                        

classes = os.listdir("drive/MyDrive/Lung Disease Detection/combined_test")
matrix = {
    "actual_diseasedxray_pred_dieasedxray":0,      # not_diseased  tp
    "actual_diseasedxray_pred_notdieasedxray":0,      # not_diseased  tp
    "actual_diseasedxray_pred_dieasedctscan":0,      # not_diseased  tp
    "actual_diseasedxray_pred_notdieasedctscan":0,      # not_diseased  tp
    "actual_diseasedctscan_pred_dieasedctscan":0,      # not_diseased  tp
    "actual_diseasedctscan_pred_notdieasedctscan":0,      # not_diseased  tp
    "actual_diseasedctscan_pred_dieasedxray":0,      # not_diseased  tp
    "actual_diseasedctscan_pred_notdieasedxray":0,      # not_diseased  tp
    "actual_notdiseasedxray_pred_notdieasedxray":0,      # not_diseased  tp
    "actual_notdiseasedxray_pred_dieasedxray":0,      # not_diseased  tp
    "actual_notdiseasedxray_pred_dieasedctscan":0,      # not_diseased  tp
    "actual_notdiseasedxray_pred_notdieasedctscan":0,      # not_diseased  tp
    "actual_notdiseasedctscan_pred_notdieasedctscan":0,      # not_diseased  tp
    "actual_notdiseasedctscan_pred_dieasedctscan":0,      # not_diseased  tp
    "actual_notdiseasedctscan_pred_notdieasedxray":0,      # not_diseased  tp
    "actual_notdiseasedctscan_pred_dieasedxray":0      # not_diseased  tp
    
}
i, s = 0, 0
for classe in classes[::]:
    i+=1
    path = f"{test_dir}/{classe}/"
    images = os.listdir(path)[::]
    for image in images:
        img = tf.keras.utils.load_img(path+image, 
                                    target_size=(img_height, img_width))
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch

        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        actual, predicted = classe, class_names[np.argmax(score)]
        if actual == "diseased_xray" and predicted == "diseased_xray":
            matrix["actual_diseasedxray_pred_dieasedxray"] += 1

        elif actual == "diseased_xray" and predicted == "not_diseased_xray":
            matrix["actual_diseasedxray_pred_notdieasedxray"] += 1

        elif actual == "diseased_xray" and predicted == "diseased_ctscan":
            matrix["actual_diseasedxray_pred_dieasedctscan"] += 1

        elif actual == "diseased_xray" and predicted == "not_diseased_ctscan":
            matrix["actual_diseasedxray_pred_notdieasedctscan"] += 1

        elif actual == "diseased_ctscan" and predicted == "diseased_ctscan":
            matrix["actual_diseasedctscan_pred_dieasedctscan"] += 1

        elif actual == "diseased_ctscan" and predicted == "not_diseased_ctscan":
            matrix["actual_diseasedctscan_pred_notdieasedctscan"] += 1

        elif actual == "diseased_ctscan" and predicted == "diseased_xray":
            matrix["actual_diseasedctscan_pred_dieasedxray"] += 1

        elif actual == "diseased_ctscan" and predicted == "not_diseased_xray":
            matrix["actual_diseasedctscan_pred_notdieasedxray"] += 1

        elif actual == "not_diseased_xray" and predicted == "not_diseased_xray":
            matrix["actual_notdiseasedxray_pred_notdieasedxray"] += 1

        elif actual == "not_diseased_xray" and predicted == "diseased_xray":
            matrix["actual_notdiseasedxray_pred_dieasedxray"] += 1

        elif actual == "not_diseased_xray" and predicted == "diseased_ctscan":
            matrix["actual_notdiseasedxray_pred_dieasedctscan"] += 1

        elif actual == "not_diseased_xray" and predicted == "not_diseased_ctscan":
            matrix["actual_notdiseasedxray_pred_notdieasedctscan"] += 1

        elif actual == "not_diseased_ctscan" and predicted == "not_diseased_ctscan":
            matrix["actual_notdiseasedctscan_pred_notdieasedctscan"] += 1

        elif actual == "not_diseased_ctscan" and predicted == "diseased_ctscan":
            matrix["actual_notdiseasedctscan_pred_dieasedctscan"] += 1

        elif actual == "not_diseased_ctscan" and predicted == "not_diseased_xray":
            matrix["actual_notdiseasedctscan_pred_notdieasedxray"] += 1

        elif actual == "not_diseased_ctscan" and predicted == "diseased_xray":
            matrix["actual_notdiseasedctscan_pred_dieasedxray"] += 1
     

        if s%100 == 0:
            print(f"{round(s*100/len(os.listdir(test_dir)))}%")
        s +=1
        
        print(
           " prediction: {} and ground truth {} : {:.2f} ."
           .format(class_names[np.argmax(score)], classe, 100 * np.max(score)))


0%
 prediction: diseased_xray and ground truth not_diseased_xray : 46.41 .
 prediction: diseased_xray and ground truth not_diseased_xray : 45.88 .
 prediction: not_diseased_xray and ground truth not_diseased_xray : 47.54 .
 prediction: diseased_xray and ground truth not_diseased_xray : 47.53 .
 prediction: not_diseased_xray and ground truth not_diseased_xray : 46.16 .
 prediction: diseased_ctscan and ground truth not_diseased_xray : 27.67 .
 prediction: not_diseased_xray and ground truth not_diseased_xray : 47.52 .
 prediction: not_diseased_xray and ground truth not_diseased_xray : 47.45 .
 prediction: diseased_xray and ground truth not_diseased_xray : 40.89 .
 prediction: diseased_ctscan and ground truth not_diseased_xray : 33.94 .
 prediction: not_diseased_xray and ground truth not_diseased_xray : 47.47 .
 prediction: not_diseased_xray and ground truth not_diseased_xray : 47.51 .
 prediction: not_diseased_xray and ground truth not_diseased_xray : 38.48 .
 prediction: diseased_ctscan 

In [73]:
matrix

{'actual_diseasedctscan_pred_dieasedctscan': 60,
 'actual_diseasedctscan_pred_dieasedxray': 0,
 'actual_diseasedctscan_pred_notdieasedctscan': 24,
 'actual_diseasedctscan_pred_notdieasedxray': 0,
 'actual_diseasedxray_pred_dieasedctscan': 2,
 'actual_diseasedxray_pred_dieasedxray': 384,
 'actual_diseasedxray_pred_notdieasedctscan': 0,
 'actual_diseasedxray_pred_notdieasedxray': 4,
 'actual_notdiseasedctscan_pred_dieasedctscan': 0,
 'actual_notdiseasedctscan_pred_dieasedxray': 0,
 'actual_notdiseasedctscan_pred_notdieasedctscan': 84,
 'actual_notdiseasedctscan_pred_notdieasedxray': 0,
 'actual_notdiseasedxray_pred_dieasedctscan': 26,
 'actual_notdiseasedxray_pred_dieasedxray': 125,
 'actual_notdiseasedxray_pred_notdieasedctscan': 1,
 'actual_notdiseasedxray_pred_notdieasedxray': 82}

In [74]:
table = [
         ["x", "not_diseased_xray_pred", "diseased_xray_pred","not_diseased_ctscan_pred","diseased_ctscan_pred"],
         ["not_diseased_xray_actual", f"{matrix['actual_notdiseasedxray_pred_notdieasedxray']}",f"{matrix['actual_notdiseasedxray_pred_dieasedxray']}",f"{matrix['actual_notdiseasedxray_pred_notdieasedctscan']}",f"{matrix['actual_notdiseasedxray_pred_dieasedctscan']}"],
         ["diseased_xray_actual", f"{matrix['actual_diseasedxray_pred_notdieasedxray']}",f"{matrix['actual_diseasedxray_pred_dieasedxray']}",f"{matrix['actual_diseasedxray_pred_notdieasedctscan']}",f"{matrix['actual_diseasedxray_pred_dieasedctscan']}"],
         ["not_diseased_ctscan_actual", f"{matrix['actual_notdiseasedctscan_pred_notdieasedxray']}",f"{matrix['actual_notdiseasedctscan_pred_dieasedxray']}",f"{matrix['actual_notdiseasedctscan_pred_notdieasedctscan']}",f"{matrix['actual_notdiseasedctscan_pred_dieasedctscan']}"],
         ["diseased_ctscan_actual", f"{matrix['actual_diseasedctscan_pred_notdieasedxray']}",f"{matrix['actual_diseasedctscan_pred_dieasedxray']}",f"{matrix['actual_diseasedctscan_pred_notdieasedctscan']}",f"{matrix['actual_diseasedctscan_pred_dieasedctscan']}"]
]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒════════════════════════════╤══════════════════════════╤══════════════════════╤════════════════════════════╤════════════════════════╕
│ x                          │   not_diseased_xray_pred │   diseased_xray_pred │   not_diseased_ctscan_pred │   diseased_ctscan_pred │
╞════════════════════════════╪══════════════════════════╪══════════════════════╪════════════════════════════╪════════════════════════╡
│ not_diseased_xray_actual   │                       82 │                  125 │                          1 │                     26 │
├────────────────────────────┼──────────────────────────┼──────────────────────┼────────────────────────────┼────────────────────────┤
│ diseased_xray_actual       │                        4 │                  384 │                          0 │                      2 │
├────────────────────────────┼──────────────────────────┼──────────────────────┼────────────────────────────┼────────────────────────┤
│ not_diseased_ctscan_actual │                        0